In [42]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, ElasticNet, LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from scipy.stats import pearsonr
import individual_prediction
import enformer_evaluation

# Running Enformer for Inividual Prediction

In [ ]:
predictions = individual_prediction.run_enformer("GTEX-1JMQI")

In [10]:
predictions = pd.read_pickle('./data_/sum_stats_GTEX-1JMQI')
predictions

,output,true,mean_prediction,gene_exp_tracks,blood_mean
0,"[[0.084296055, 0.08817325, 0.10868081, 0.08987...",0.706494,1.001050,0.072144,0.094448
1,"[[0.079454444, 0.08264593, 0.10211448, 0.08053...",1.382185,0.946958,0.074200,0.085729
2,"[[0.0372191, 0.036553897, 0.028942728, 0.05171...",0.433461,0.726118,0.030193,0.054167
3,"[[0.10508142, 0.112968825, 0.1411426, 0.095036...",-1.442962,1.004281,0.240938,0.327627
4,"[[0.18740812, 0.18950415, 0.23729949, 0.127890...",-1.396898,1.161463,0.303934,0.470002
...,...,...,...,...,...
569,"[[0.32434317, 0.35366538, 0.48401946, 0.182931...",-0.462371,1.438412,0.329137,0.317762
570,"[[0.28047597, 0.3232522, 0.44591138, 0.1704778...",1.034199,1.359313,0.438184,0.319138
571,"[[0.30443397, 0.3588802, 0.5324826, 0.1694706,...",-0.445806,1.295028,0.306191,0.214336
572,"[[0.19074407, 0.21617892, 0.3565924, 0.0997722...",1.015253,0.684483,0.109743,0.078812


# Transforming Enformer Output

In [6]:
files = ['./data_/sum_stats_gtex111ys.pkl', './data_/sum_stats_gtex113jc', \
          './data_/sum_stats_gtex117xs.pkl', './data_/sum_stats_gtexzvt4', \
          './data_/sum_stats_gtexrws6.pkl']
true, predict_mean = enformer_evaluation.transform(files)

# Fitting Stacked Model

In [20]:
predict_mean

,0,1,2,3,4
0,-1.377952,-1.377952,-1.366975,-1.376278,-1.379207
1,-1.539162,-1.539162,-1.531238,-1.538084,-1.539261
2,-2.204406,-2.204406,-2.193338,-2.202546,-2.205193
3,-1.368387,-1.368387,-1.367935,-1.369656,-1.369040
4,-0.899897,-0.899897,-0.901556,-0.902284,-0.902252
...,...,...,...,...,...
569,-0.072945,-0.072945,-0.073056,-0.080115,-0.075921
570,-0.309928,-0.309928,-0.309494,-0.314916,-0.311546
571,-0.503441,-0.503441,-0.503285,-0.507935,-0.497926
572,-2.319106,-2.319106,-2.320589,-2.320122,-2.321883


In [23]:
X_train = predict_mean.iloc[:,1:].values.T
y_train = true.iloc[:,1:].values.T

X_test = predict_mean[['0']].values.T
y_test = true[['0']].values.T

## Ridge Regression

In [24]:
ridge = Ridge()
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
ridge_cv = GridSearchCV(ridge, param_grid, scoring='r2', cv=2)
ridge_cv.fit(X_train, y_train)

best_alpha = ridge_cv.best_params_['alpha']
best_ridge_model = ridge_cv.best_estimator_

print(f"Best Alpha for Ridge: {best_alpha}")

Best Alpha for Ridge: 0.001


In [25]:
ridge = Ridge(alpha=0.001)
ridge.fit(X_train, y_train)

ridge_preds = ridge.predict(X_test)

## Random Forest Regressor

In [26]:
rf = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['sqrt', 'log2']
}

rf_grid = GridSearchCV(rf, param_grid, scoring='r2', cv=2, n_jobs=-1)
rf_grid.fit(X_train, y_train)
best_rf_params = rf_grid.best_params_
best_rf_model = rf_grid.best_estimator_

print(f"Best Parameters for Random Forest: {best_rf_params}")

Best Parameters for Random Forest: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


In [27]:
rf = RandomForestRegressor(max_depth=10, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, n_estimators=100)
rf.fit(X_train, y_train)
rf_predicted = rf.predict(X_test)

## Polynomial

In [28]:
poly_model = make_pipeline(PolynomialFeatures(degree=2), Ridge(alpha=0.1))
poly_model.fit(X_train, y_train)
poly_adjusted_predictions = poly_model.predict(X_test)

## Elastic Net

In [36]:
elastic_net = ElasticNet()
elastic_net_param_grid = {'alpha': np.logspace(-3, 3, 50), 'l1_ratio': np.linspace(0.1, 0.9, 10)}
elastic_net_search = RandomizedSearchCV(elastic_net, elastic_net_param_grid, n_iter=20, scoring='r2', cv=2, random_state=42)
elastic_net_search.fit(X_train, y_train)
elastic_net_best = elastic_net_search.best_estimator_
print(f"Best Parameters for Elastic Net: {elastic_net_search.best_params_}")

Best Parameters for Elastic Net: {'l1_ratio': np.float64(0.1), 'alpha': np.float64(0.002329951810515372)}


In [44]:
elastic_net = ElasticNet(alpha=0.002329951810515372, l1_ratio=0.1)
elastic_net.fit(X_train, y_train)
pred_elastic_net = elastic_net.predict(X_test)

## Stacked Model

In [46]:
stack_train = np.column_stack((
    ridge.predict(X_train), 
    rf.predict(X_train), 
    poly_model.predict(X_train), 
    elastic_net.predict(X_train)
))

stack_test = np.column_stack((
    ridge.predict(X_test), 
    rf.predict(X_test),
    poly_model.predict(X_test),
    elastic_net.predict(X_test)
))

In [47]:
meta_model = XGBRegressor(
    n_estimators=100,
    learning_rate=0.005,
    max_depth=5,
    subsample=0.7,         
    colsample_bytree=0.7, 
    colsample_bylevel=0.8, 
    reg_lambda=1,    
    reg_alpha=0.5, 
    random_state=None      
)
meta_model.fit(stack_train, y_train)
stacked_pred = meta_model.predict(stack_test)
stacked_score = pearsonr(y_test.flatten(), stacked_pred.flatten())[0]
print(f"Stacked Model Pearson Correlation: {stacked_score:.4f}")

Stacked Model Pearson Correlation: -0.3030


In [48]:
p_value = pearsonr(y_test.flatten(), stacked_pred.flatten())[1]

In [49]:
p_value

np.float64(1.181424274747606e-13)